In [1]:
#load necessary libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import datetime
import time
import os

In [13]:
# 读取CSV文件
df = pd.read_csv('Valid_NoDuplicate_djl.csv')

# 计算总行数
total_rows = df.shape[0]

# 计算需要的子数据框数量
num_subframes = (total_rows // 10) + (1 if total_rows % 10 != 0 else 0)

# 分割数据框并保存
for i in range(num_subframes):
    start_row = i * 10
    end_row = min(start_row + 10, total_rows)
    subframe = df.iloc[start_row:end_row]
    subframe.to_csv(f'users/subframe_{i+1}.csv', index=False)

print(f'{num_subframes} subframes have been created.')


58 subframes have been created.


In [2]:
def login():

    # 获取登录页面
    response = session.get(login_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 提取必要的表单数据，假设CSRF token字段名称是 'csrfmiddlewaretoken'
    csrf_token = soup.find('input', {'name': 'csrfmiddlewaretoken'})['value']

    #TODO：username_or_email/password填入自己的
    # 构造登录请求的数据
    login_data = {
        'csrfmiddlewaretoken': csrf_token,
        'next': '/login?next=/user/jolesreal/library/albums',
        'username_or_email': 'dingjianle@gmail.com',
        'password': 'Djl20030307.....',
        'submit': '登录'
    }

    # 发送登录请求
    login_response = session.post(login_url, data=login_data, headers=headers)
    return login_response

In [10]:
def getOnePage(page_num, name,personal_data):
    print("page_num:{}".format(page_num))
    page_url = f'https://www.last.fm/user/{name}/library/albums?page={page_num}'
    page_response = session.get(page_url, headers=headers)
    page_soup = BeautifulSoup(page_response.text, 'html.parser')
    target_html = page_soup.find('tbody', {'data-playlisting-add-entries': True})

    if target_html:
        chartlist_rows = target_html.find_all('tr', class_='chartlist-row')


        for row in chartlist_rows:
            chartlist_index = row.find('td', class_='chartlist-index').get_text(strip=True)
            album_name = row.find('td', class_='chartlist-name').find('a').get_text(strip=True)
            if album_name not in song_names.values:
                continue
            artist_name = row.find('td', class_='chartlist-artist').find('a').get_text(strip=True)
            play_count = row.find('span', class_='chartlist-count-bar-value').get_text(strip=True).split("s")[0].replace(",","")

            personal_data.append({
                "user":name,
                "rank": chartlist_index,
                "album_name": album_name,
                "artist": artist_name,
                "play_count": play_count,
            })
        return page_num+1
            
    else:
        print("Page {} Once again".format(page_num))
        time.sleep(20)
        return page_num


In [11]:
def getOnePerson(name):
    personal_data = []
    page_url = f'https://www.last.fm/user/{name}/library/albums'
    
    while True:
        page_response = session.get(page_url, headers=headers)
        if page_response.status_code != 200:
            print("Error: Unable to fetch page for {}. Status code: {}".format(name, page_response.status_code))
            time.sleep(5)  # 等待5秒后重试
            continue
        
        soup = BeautifulSoup(page_response.text, 'html.parser')
        metadata_element = soup.find('p', {'class': 'metadata-display', 'data-top-item-count': ''})
        
        if metadata_element:
            element = metadata_element.get_text(strip=True).replace(",", "")
            if int(element) % 50 == 0:
                num = int(int(element) / 50)
            else:
                num = int(int(element) / 50) + 1
            print("num:{}".format(num))
            page_num = 1
            while page_num <= num:
                page_num = getOnePage(page_num, name, personal_data)
            print("{} Success, total {}".format(name, len(personal_data)))
            all_data.extend(personal_data)
            return  # 成功后退出循环
        else:
            print("Metadata not found for {}. Retrying...".format(name))
            time.sleep(5)  # 等待5秒后重试

In [ ]:
# 创建一个会话对象
session = requests.Session()
#TODO：变成专辑的CSV
df = pd.read_csv('grouped_data.csv')
song_names = df['name']

login_url = 'https://www.last.fm/login'

headers = {
    'Referer': login_url,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
login_response = login()
# 检查是否登录成功
if 'incorrect' not in login_response.text:
    for i in range(58, 50, -1):
        # TODO:换成你的对应的用户的CSV
        df = pd.read_csv(f'users/subframe_{i}.csv')
        names = df["user"].values
        names = set(names)

        all_data = []
        
        for name in names:
            getOnePerson(name)
            

        # 创建结果文件夹（如果不存在）
        result_dir = 'result'
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
        
        # 保存结果到CSV文件
        result_df = pd.DataFrame(all_data)
        result_df.to_csv(f'{result_dir}/subframe_{i}.csv', index=False)
        print("Finished")
else:
    print('登录失败，请检查用户名和密码')

num:161
page_num:1
page_num:2
page_num:3
page_num:4
page_num:5
page_num:6
page_num:7
page_num:8
page_num:9
page_num:10
page_num:11
page_num:12
page_num:13
page_num:14
page_num:15
page_num:16
page_num:17
page_num:18
page_num:19
Page 19 Once again
page_num:19
Page 19 Once again
page_num:19
page_num:20
page_num:21
page_num:22
page_num:23
page_num:24
page_num:25
page_num:26
page_num:27
page_num:28
page_num:29
page_num:30
page_num:31
page_num:32
page_num:33
page_num:34
page_num:35
page_num:36
page_num:37
page_num:38
page_num:39
Page 39 Once again
page_num:39
page_num:40
page_num:41
page_num:42
page_num:43
page_num:44
page_num:45
page_num:46
page_num:47
page_num:48
page_num:49
page_num:50
page_num:51
page_num:52
page_num:53
page_num:54
page_num:55
page_num:56
Page 56 Once again
page_num:56
Page 56 Once again
page_num:56
page_num:57
page_num:58
page_num:59
page_num:60
page_num:61
page_num:62
page_num:63
page_num:64
page_num:65
page_num:66
page_num:67
page_num:68
page_num:69
page_num:70
page_n

In [28]:
df = pd.DataFrame(all_data)
#TODO：变成自己的CSV
df

,user,rank,album_name,artist,play_count
0,xylonaaa,3,Phases,Chase Atlantic,2915
1,xylonaaa,15,A N N I V E R S A R Y,Bryson Tiller,466
2,xylonaaa,22,PARTYNEXTDOOR,PARTYNEXTDOOR,302
3,xylonaaa,33,?,XXXTENTACION,170
4,xylonaaa,43,All I Want Is You,Miguel,124
...,...,...,...,...,...
80,xylonaaa,"1,010",iSouljaBoyTellEm,Soulja Boy,1
81,xylonaaa,"1,031",greedy,Tate McRae,1
82,xylonaaa,"1,059",The Feels,TWICE,1
83,xylonaaa,"1,081",Fifty Shades of Grey (Original Motion Picture ...,Various Artists,1
